In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import math
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Preprocessing

In [ ]:
dataset = pd.read_csv("../input/tglf-new-coefficients/TAML_227_test.csv")#TAML_227_actually_logged.csv")
dataset= dataset.dropna()
#dataset = dataset[dataset['cc'] > 0.3]

dataset.shape
dataset.head()

In [ ]:
x = dataset.iloc[:, 0:13]
# x["RLTS_1^2"]=x["RLTS_1"]**2
# x.drop("RLTS_1", axis=1)

#inverse
#inv_x = x.join((1/(x+0.001)).add_prefix('inv_'))
#x = inv_x

y = dataset.iloc[:, 13:16]

# for i in range(len(x.columns)):
#     #new_col = "log10(abs("+x.columns[i]+"))"
#     x =x.rename(columns={x.columns[i]:new_col})
    
x.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 2)

y_cols = list(y.columns)
x_cols = list(x.columns)
print(x_cols)
print(y_cols)

XGBoost (for base accuracy)

In [ ]:
from xgboost import XGBRegressor

print(y_cols)
for i in y_cols:
    xgb = XGBRegressor()
    xgb.fit(x_train, y_train[i])

    y_pred = xgb.predict(x_test)

    from sklearn.metrics import r2_score
    print(i,r2_score(y_test[i],y_pred))
    
    cost = sum((y_pred - y_test[i])**2) / sum((y_test[i] - np.mean(y_pred))**2)
    print("Cost",cost)

# PolyTrain Function

Getting Coefficients of A Polynomial (it seems degree=2 produces the most accurate results)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, HuberRegressor, SGDRegressor, Ridge, Lasso, ElasticNet
import matplotlib.pyplot as plt
import matplotlib

# max_r_score = 0
def poly_train(x_train, x_test, y_train, y_test, degree, graph=False):
    poly = PolynomialFeatures(degree = degree)
    x_poly = poly.fit_transform(x_train)
    poly.fit(x_poly, y_train)
    
    #increase iterations, epsilon
    model = LinearRegression()#HuberRegressor(epsilon=1.0,max_iter=1000,alpha=0.05)
    model.fit(x_poly, y_train)

    y_pred = model.predict(poly.fit_transform(x_test))
    
    from sklearn.metrics import r2_score, mean_squared_error
    r_score = r2_score(y_pred,y_test)
    mse = mean_squared_error(y_pred,y_test)
    cost = sum(abs(y_pred - y_test))
    
    if graph:
        max_r_score = r_score
        print("L1 Error: ",cost)

        print("R-score: ",r_score)
        
        print("MSE: ",mse)

        coefs = list(model.coef_)[1:]
        intercept = str(model.intercept_)
        features = poly.get_feature_names(x_train.columns)[1:]
        for feat in range(len(features)):
            split_feats=features[feat].split(' ')
            if len(split_feats) > 1:
                features[feat]=split_feats[0]+'*'+split_feats[1]
            if "^" in features[feat]:
                features[feat]=features[feat][:-2]+"**2"
            
        equation = "=10**("
        for i in range(len(coefs)):
            equation += str(coefs[i])+"*"+features[i] +" + " #"& \n"
        equation += intercept
        equation += ")"
        print("Equation: ",equation)
        
        plt.ylabel('analytic')
        plt.xlabel('NN')

        plt.hist2d(y_test, y_pred, bins=100, cmap='viridis', norm=matplotlib.colors.LogNorm())
        plt.plot([-1.5, 2],[-1.5, 2], color='red', linewidth=3)
        
#         plt.xlim(-1.5,2)
#         plt.ylim(-1.5,2)
#         plt.xlim(-1, 2)
#         plt.ylim(-1, 2)
        plt.show()
    return r_score

Base polynomial accuracy

In [ ]:
max_r_score = 0

print("aa")
poly_train(x_train, x_test, y_train['aa'], y_test['aa'], 1, graph=True)

print("bb")
poly_train(x_train, x_test, y_train['bb'], y_test['bb'], 1, graph=True)

print("cc")
poly_train(x_train, x_test, y_train['cc'], y_test['cc'], 1, graph=True)

Bruteforce Combination of all variables

In [ ]:
import itertools
def feature_selection(coefficient):
    columns = x.columns.tolist() 
    print(columns)
    features = []
    complexity = []
    scores = []
    
    print("Best combination")
    for k in range(1,5):
        print("Trying",str(k))
        combinations = itertools.combinations(columns, k)
        max_r_score = -1e9
        for combination in combinations:
            r_score = poly_train(x_train[list(combination)], x_test[list(combination)], y_train[coefficient], y_test[coefficient], 1)
            if r_score > max_r_score:
                max_r_score = r_score
                features = combination
        scores.append(1-max_r_score)
        complexity.append(k)
        print(sorted(features))
    
    print("\nSequentially removed features")
    scores_sequentially_subtracted = []
    feature = ""
    selected_features_subtraction=[]
    for k in range(1,5):
        print("Trying with -",k,"features")
        max_r_score = -1e9
        for col in columns:
            cols = set(columns) - set([col])
            r_score = poly_train(x_train[cols], x_test[cols], y_train[coefficient], y_test[coefficient], 1)
            if r_score > max_r_score:
                max_r_score = r_score
                feature = col
        scores_sequentially_subtracted.insert(0,1-max_r_score)
        selected_features_subtraction.append(feature)
        columns.remove(feature)
        print(sorted(columns))
    
    print("\nSequentially added features")
    columns = x.columns.tolist() 
    scores_sequentially_added = []
    feature = ""
    selected_features=[]
    for k in range(1,5):
        print("Trying with",k,"features")
        max_r_score = -1e9
        for col in columns:
            cols = selected_features + [col]
            r_score = poly_train(x_train[cols], x_test[cols], y_train[coefficient], y_test[coefficient], 1)
            if r_score > max_r_score:
                max_r_score = r_score
                feature = col
        scores_sequentially_added.append(1-max_r_score)
        selected_features.append(feature)
        columns.remove(feature)
        print(sorted(selected_features))
    
    
    
    plt.title(coefficient)
    plt.xlabel("# of Terms")
    plt.ylabel("1-r^2")
    plt.plot(complexity,scores, color='blue', marker='o', label="Best combination")
    plt.plot(complexity,scores_sequentially_subtracted, color='red', marker='o', label="Sequentially removed features")
    plt.plot(complexity,scores_sequentially_added, color='green', marker='o', label="Sequentially added features")
    plt.legend()

In [ ]:
feature_selection("aa")

In [ ]:
feature_selection("bb")

In [ ]:
feature_selection("cc")

Now let's take a closer look at the features we chose from the output of the feature_selection function

In [ ]:
chosen_features_aa = ['DELTA_LOC', 'P_PRIME_LOC', 'RLTS_1', 'TAUS_2', 'VEXB_SHEAR', 'XNUE']
poly_train(x_train[list(chosen_features_aa)], x_test[list(chosen_features_aa)], y_train['aa'], y_test['aa'], 1,True)

In [ ]:
chosen_features_bb = ['DEBYE', 'DELTA_LOC', 'Q_PRIME_LOC', 'RLNS_2', 'RLTS_1', 'VPAR_1']
poly_train(x_train[list(chosen_features_bb)], x_test[list(chosen_features_bb)], y_train['bb'], y_test['bb'], 1,True)

In [ ]:
chosen_features_cc = ['DELTA_LOC', 'RLTS_1', 'RLTS_2', 'TAUS_2', 'XNUE']
poly_train(x_train[list(chosen_features_cc)], x_test[list(chosen_features_cc)], y_train['cc'], y_test['cc'], 1,True)

# Feature Selection Function

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectKBest
import matplotlib.pyplot as plt

def getFeatures(column):
    #Extra Trees
    np.seterr(divide='ignore', invalid='ignore')
    feature_tree = ExtraTreesRegressor()
    feature_tree.fit(x,y[column])

    feat_importances_tree = pd.Series(feature_tree.feature_importances_, index=x.columns)
    tree_features = feat_importances_tree.nlargest(13)
#     tree_features.plot(kind='barh')
#     plt.show()
    
    #KBest
    KBest = SelectKBest()
    feature_kbest = KBest.fit(x,y[column])
    feat_importances_kbest = pd.Series(feature_kbest.scores_, index=x.columns)
    kbest_features = feat_importances_kbest.nlargest(13)
#     kbest_features.plot(kind='barh')
#     plt.show()
    return kbest_features.keys(), tree_features.keys()

getFeatures("aa")

In [ ]:
def generateEquations(complexity, column, degrees):
    features = 13-complexity
    kbest_feats, tree_feats = getFeatures(column)

    kbest_cols = []
    for i in kbest_feats[:-features]:
        kbest_cols.append(i)
        for degree in degrees:
            poly_train(x_train[kbest_cols], x_test[kbest_cols], y_train[column], y_test[column], degree, graph=True)
            
    tree_cols = []
    for j in tree_feats[:-features]:
        tree_cols.append(j)
        for degree in degrees:
            poly_train(x_train[tree_cols], x_test[tree_cols], y_train[column], y_test[column], degree, graph=True)

# Models with less complexity and printed coefficients

In [ ]:
#aa
generateEquations(6, "aa", [1])

In [ ]:
#bb
generateEquations(8, "bb", [1])

In [ ]:
#cc
generateEquations(12, "cc", [1])